# Summarizing with LangChain and OpenAI

##### 1) When you see `from langchain.chat_models import ChatOpenAI` change it to `from langchain_openai import ChatOpenAI`

##### 2) Also, instead of `answer = chain.run(q)` use `answer = chain.invoke(q)`

In [ ]:
import os 
from dotenv import load_dotenv, find_dotenv 

load_dotenv(find_dotenv(), override=True)

### 1. Summarizing Using a Basic Prompt

In [1]:
from langchain_openai import ChatOpenAI 
from langchain.schema import AIMessage, HumanMessage, SystemMessage 

In [2]:
text= r"""
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

In [3]:
messages=[
    SystemMessage(content="You are an expert copywriter with expertize in sumarizing documents."), 
    HumanMessage(content=f"Please provide a short and concise summary of the following text:\n TEXT: {text}")
] 

llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [4]:
llm.get_num_tokens(text)

238

In [5]:
summary_output=llm.invoke(messages) 

print(summary_output) 
print("-"*100) 
print(summary_output.content)

content="Mojo is a new programming language that combines Python's usability with C's performance, enabling advanced programmability of AI hardware and models. It aims to bridge the gap between research and production by offering faster, portable code that seamlessly integrates with the Python ecosystem. The language was developed by Modular to simplify programming across the ML/AI stack and optimize performance on host CPUs, which play a crucial role in AI operations." response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 278, 'total_tokens': 360}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7bdcca1d-5801-4c91-8ed8-6486cf235fa0-0' usage_metadata={'input_tokens': 278, 'output_tokens': 82, 'total_tokens': 360}
----------------------------------------------------------------------------------------------------
Mojo is a new programming language that combines Python's usability with C's performan

### 2. Summarizing Using Prompt Templates (Length must be Lower than Model's Allowed Token Length)

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
template="""   
Write a concise and short summary of the following text: 
TEXT: `{text}`
Translate the summary to {language}. 
"""

prompt=PromptTemplate(
    input_variables=["text", "language"], 
    template=template
    )

In [9]:
llm.get_num_tokens(prompt.format(text=text, language="English"))

261

In [13]:
chain=LLMChain(llm=llm, prompt=prompt) 

response=chain.invoke({"text": text, "language": "English"}) 

print(response) 
print("-"*100) 
print(response["text"])

{'text': 'Mojo is a new programming language that combines the usability of Python with the performance of C, allowing for unparalleled programmability of AI hardware and extensibility of AI models. It aims to bridge the gap between research and production by offering faster code than C and seamless integration with the Python ecosystem. Mojo was developed by Modular to simplify programming across the entire ML/AI stack and make use of the host CPU as an important accelerator for various operations.', 'language': 'English'}
----------------------------------------------------------------------------------------------------
Mojo is a new programming language that combines the usability of Python with the performance of C, allowing for unparalleled programmability of AI hardware and extensibility of AI models. It aims to bridge the gap between research and production by offering faster code than C and seamless integration with the Python ecosystem. Mojo was developed by Modular to simpli

In [14]:
chain=LLMChain(llm=llm, prompt=prompt) 

response=chain.invoke({"text": text, "language": "Spanish"}) 

print(response) 
print("-"*100) 
print(response["text"])

{'text': 'Mojo es un nuevo lenguaje de programación que combina la usabilidad de Python con el rendimiento de C, permitiendo una programabilidad sin igual del hardware de IA y la extensibilidad de los modelos de IA. Con Mojo, se puede escribir código portátil más rápido que C y trabajar de forma integrada con el ecosistema de Python. Este lenguaje surge de la necesidad de simplificar la programación en el ámbito de la inteligencia artificial y la máquina aprendizaje, facilitando la transición de la investigación a la producción. Además, se destaca la importancia de los aceleradores de CPU en el procesamiento de datos y operaciones de IA.', 'language': 'Spanish'}
----------------------------------------------------------------------------------------------------
Mojo es un nuevo lenguaje de programación que combina la usabilidad de Python con el rendimiento de C, permitiendo una programabilidad sin igual del hardware de IA y la extensibilidad de los modelos de IA. Con Mojo, se puede esc

### 3. Summarizing using StuffDocumentsChain